Credit Scoring Fairness Criteria

In [1]:
#Load the dataset

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

Load the dataset

In [3]:
import os
BANK_PATH = os.path.join(os.getcwd(), './bankfullclean.csv') # Assign this to the location of the dataset
print(os.getcwd())
print(BANK_PATH)

# 5254 number of rows have 'biY' == 1
# 39787 number of rows have 'biY' == 0


/home/mj33/LLMs_Program_Synthesis_Eval/example_code/Python/Credit_Scoring_Fairness
/home/mj33/LLMs_Program_Synthesis_Eval/example_code/Python/Credit_Scoring_Fairness/./bankfullclean.csv


In [4]:
bank = pd.read_csv(BANK_PATH)
df = pd.DataFrame(bank)
df.head()

,Unnamed: 0,age,job,marital,education,default,balance,housing,loan,day,...,y,Fmonth,log_duration,log_campaign,catAge,biMarital,biDefault,biHousing,biLoan,biY
0,1,58,management,married,tertiary,no,2143,yes,no,5,...,no,may,5.564520,0.0,>=50,1,0,1,0,0
1,2,44,technician,single,secondary,no,29,yes,no,5,...,no,may,5.017280,0.0,40-50,0,0,1,0,0
2,3,33,entrepreneur,married,secondary,no,2,yes,yes,5,...,no,may,4.330733,0.0,30-40,1,0,1,1,0
3,4,47,blue-collar,married,secondary,no,1506,yes,no,5,...,no,may,4.521789,0.0,40-50,1,0,1,0,0
4,6,35,management,married,tertiary,no,231,yes,no,5,...,no,may,4.934474,0.0,30-40,1,0,1,0,0


In [5]:
df =  df.sample(n=500, random_state=1234).reset_index(drop=True)
df

,Unnamed: 0,age,job,marital,education,default,balance,housing,loan,day,...,y,Fmonth,log_duration,log_campaign,catAge,biMarital,biDefault,biHousing,biLoan,biY
0,25108,42,housemaid,divorced,tertiary,no,364,no,no,18,...,no,nov,3.951244,0.000000,40-50,0,0,0,0,0
1,20138,60,retired,married,primary,no,115,no,no,11,...,no,aug,4.406719,0.693147,>=50,1,0,0,0,0
2,33895,47,blue-collar,married,secondary,no,1230,yes,no,30,...,yes,apr,6.021023,0.000000,40-50,1,0,1,0,1
3,23385,53,technician,divorced,secondary,no,28,no,no,27,...,no,aug,5.680173,2.197225,>=50,0,0,0,0,0
4,12293,47,retired,married,primary,no,-468,no,yes,25,...,no,jun,5.590987,0.000000,40-50,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,10559,60,admin.,married,secondary,no,504,no,no,16,...,no,jun,5.181784,1.386294,>=50,1,0,0,0,0
496,34110,54,management,married,tertiary,no,1014,no,yes,30,...,yes,apr,5.480639,0.000000,>=50,1,0,0,1,1
497,18042,46,technician,divorced,secondary,no,2862,yes,yes,30,...,no,jul,4.882802,1.098612,40-50,0,0,1,1,0
498,2330,44,technician,single,tertiary,no,166,yes,no,13,...,no,may,5.587249,1.098612,40-50,0,0,1,0,0


In [6]:


dummy = df[['catAge', 'job', 'marital','education','balance','day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'Fmonth', 'biDefault', 'biHousing', 'biLoan']]
# dummy.columns.values
dummy = pd.get_dummies(data=dummy)
# dummy.head()
dummy.columns.values
# df['marital'].unique()

# df['biY'].replace({0: False, 1: True}, inplace=True)
# print(df['biY'])


array(['balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'biDefault', 'biHousing', 'biLoan', 'catAge_30-40', 'catAge_40-50',
       'catAge_<=30', 'catAge_>=50', 'job_admin.', 'job_blue-collar',
       'job_entrepreneur', 'job_housemaid', 'job_management',
       'job_retired', 'job_self-employed', 'job_services', 'job_student',
       'job_technician', 'job_unemployed', 'marital_divorced',
       'marital_married', 'marital_single', 'education_primary',
       'education_secondary', 'education_tertiary', 'month_apr',
       'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul',
       'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
       'month_sep', 'Fmonth_apr', 'Fmonth_aug', 'Fmonth_dec',
       'Fmonth_feb', 'Fmonth_jan', 'Fmonth_jul', 'Fmonth_jun',
       'Fmonth_mar', 'Fmonth_may', 'Fmonth_nov', 'Fmonth_oct'],
      dtype=object)

Machine Learning Classification Model

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
def get_naive_dataset(dataset):
    dataset = dataset.sample(frac=1, random_state=1234).reset_index(drop=True)
    X = dummy
    y = dataset['biY']
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
    
    return (x_train, y_train), (x_test, y_test)


In [9]:
(x_train, y_train), (x_test, y_test) = get_naive_dataset(df)
x_train.shape
x_test.shape

(100, 53)

In [10]:
x_train.columns.values

array(['balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'biDefault', 'biHousing', 'biLoan', 'catAge_30-40', 'catAge_40-50',
       'catAge_<=30', 'catAge_>=50', 'job_admin.', 'job_blue-collar',
       'job_entrepreneur', 'job_housemaid', 'job_management',
       'job_retired', 'job_self-employed', 'job_services', 'job_student',
       'job_technician', 'job_unemployed', 'marital_divorced',
       'marital_married', 'marital_single', 'education_primary',
       'education_secondary', 'education_tertiary', 'month_apr',
       'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul',
       'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
       'month_sep', 'Fmonth_apr', 'Fmonth_aug', 'Fmonth_dec',
       'Fmonth_feb', 'Fmonth_jan', 'Fmonth_jul', 'Fmonth_jun',
       'Fmonth_mar', 'Fmonth_may', 'Fmonth_nov', 'Fmonth_oct'],
      dtype=object)

In [11]:
y_train.values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [12]:
x_test.columns.values

array(['balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'biDefault', 'biHousing', 'biLoan', 'catAge_30-40', 'catAge_40-50',
       'catAge_<=30', 'catAge_>=50', 'job_admin.', 'job_blue-collar',
       'job_entrepreneur', 'job_housemaid', 'job_management',
       'job_retired', 'job_self-employed', 'job_services', 'job_student',
       'job_technician', 'job_unemployed', 'marital_divorced',
       'marital_married', 'marital_single', 'education_primary',
       'education_secondary', 'education_tertiary', 'month_apr',
       'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul',
       'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
       'month_sep', 'Fmonth_apr', 'Fmonth_aug', 'Fmonth_dec',
       'Fmonth_feb', 'Fmonth_jan', 'Fmonth_jul', 'Fmonth_jun',
       'Fmonth_mar', 'Fmonth_may', 'Fmonth_nov', 'Fmonth_oct'],
      dtype=object)

In [13]:
y_test.values

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Naive Bayes Model

In [14]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb_pred = gnb.fit(x_train, y_train).predict(x_test)

Result

In [15]:
test_df = x_test.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(gnb_pred, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])
test_df.columns.values

array(['balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'biDefault', 'biHousing', 'biLoan', 'catAge_30-40', 'catAge_40-50',
       'catAge_<=30', 'catAge_>=50', 'job_admin.', 'job_blue-collar',
       'job_entrepreneur', 'job_housemaid', 'job_management',
       'job_retired', 'job_self-employed', 'job_services', 'job_student',
       'job_technician', 'job_unemployed', 'marital_divorced',
       'marital_married', 'marital_single', 'education_primary',
       'education_secondary', 'education_tertiary', 'month_apr',
       'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul',
       'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
       'month_sep', 'Fmonth_apr', 'Fmonth_aug', 'Fmonth_dec',
       'Fmonth_feb', 'Fmonth_jan', 'Fmonth_jul', 'Fmonth_jun',
       'Fmonth_mar', 'Fmonth_may', 'Fmonth_nov', 'Fmonth_oct', 'biY',
       'pred', 'accurate'], dtype=object)

In [16]:
from sklearn.metrics import confusion_matrix

In [17]:

confusion_matrix = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix

array([[44, 44],
       [ 7,  5]])

In [18]:
# Accuracy
# "Accuracy: ", test_df.accurate.mean()
accuracy_naive_bayes = (confusion_matrix[0][0] + confusion_matrix[1][1]) / (confusion_matrix[0][0] + confusion_matrix[0][1] + confusion_matrix[1][0] + confusion_matrix[1][1])
accuracy_naive_bayes



0.49

In [19]:
# Split dataset based on marital status
test_df.columns.values
test_married = x_test[x_test['marital_married'] == 1]
test_single = x_test[x_test['marital_single'] == 1]
test_divorced = x_test[x_test['marital_divorced'] == 1]

In [20]:
# Test on single subset
gnb_pred_single = gnb.fit(x_train, y_train).predict(test_single)

test_df = test_single.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(gnb_pred_single, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_single = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_single


array([[ 8, 20],
       [ 2,  1]])

In [21]:
accuracy_naive_bayes_single = (confusion_matrix_single[0][0] + confusion_matrix_single[1][1]) / (confusion_matrix_single[0][0] + confusion_matrix_single[0][1] + confusion_matrix_single[1][0] + confusion_matrix_single[1][1])
accuracy_naive_bayes_single

0.2903225806451613

In [22]:
# Test on married subset
gnb_pred_married = gnb.fit(x_train, y_train).predict(test_married)

test_df = test_married.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(gnb_pred_married, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_married = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_married

array([[26, 21],
       [ 5,  4]])

In [23]:
accuracy_naive_bayes_married = (confusion_matrix_married[0][0] + confusion_matrix_married[1][1]) / (confusion_matrix_married[0][0] + confusion_matrix_married[0][1] + confusion_matrix_married[1][0] + confusion_matrix_married[1][1])
accuracy_naive_bayes_married 

0.5357142857142857

In [24]:
# Test on divorced subset
gnb_pred_divorced = gnb.fit(x_train, y_train).predict(test_divorced)

test_df = test_divorced.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(gnb_pred_divorced, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_divorced = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_divorced

array([[10,  3],
       [ 0,  0]])

In [25]:
accuracy_naive_bayes_divorced = (confusion_matrix_divorced[0][0] + confusion_matrix_divorced[1][1]) / (confusion_matrix_divorced[0][0] + confusion_matrix_divorced[0][1] + confusion_matrix_divorced[1][0] + confusion_matrix_divorced[1][1])
accuracy_naive_bayes_divorced # 0.8718606782768102

0.7692307692307693

In [26]:
#Demographic Parity
# positive rate = TP + TN
PR_Naive_Bayes_married = confusion_matrix_married[0,0] +confusion_matrix_married[1,1]

PR_Naive_Bayes_married 


30

In [27]:
PR_Naive_Bayes_single = confusion_matrix_single[0,0] +confusion_matrix_single[1,1]
PR_Naive_Bayes_single 

9

In [28]:
PR_Naive_Bayes_divorced = confusion_matrix_divorced[0,0] +confusion_matrix_divorced[1,1]
PR_Naive_Bayes_divorced 

10

In [29]:
# Equalized Opportunity
#TPR = TP/TP+FN

TPR_Naive_Bayes_married = confusion_matrix_married[0,0] / (confusion_matrix_married[0,0] + confusion_matrix_married[1,0])

TPR_Naive_Bayes_married 


0.8387096774193549

In [30]:
TPR_Naive_Bayes_single = confusion_matrix_single[0,0] / (confusion_matrix_single[0,0] + confusion_matrix_single[1,0])

TPR_Naive_Bayes_single 

0.8

In [31]:
TPR_Naive_Bayes_divorced = confusion_matrix_divorced[0,0] / (confusion_matrix_divorced[0,0] + confusion_matrix_divorced[1,0])

TPR_Naive_Bayes_divorced # 0.8925619834710744

1.0

In [32]:
# Equalized Odds
#TPR = TP/TP+FN
#FNR = FN/FN+TP


FNR_Naive_Bayes_married = confusion_matrix_married[1,0] / (confusion_matrix_married[1,0] + confusion_matrix_married[0,0])

FNR_Naive_Bayes_married 



0.16129032258064516

In [33]:
FNR_Naive_Bayes_single = confusion_matrix_single[1,0] / (confusion_matrix_single[1,0] + confusion_matrix_single[0,0])

FNR_Naive_Bayes_single 

0.2

In [34]:
FNR_Naive_Bayes_divorced = confusion_matrix_divorced[1,0] / (confusion_matrix_divorced[1,0] + confusion_matrix_divorced[0,0])

FNR_Naive_Bayes_divorced 

0.0

In [35]:
# Fairness Through Unwareness
x_train_unawareness = x_train.copy()
x_train_unawareness
x_train_unawareness.drop('marital_married', inplace=True, axis=1)
x_train_unawareness.drop('marital_single', inplace=True, axis=1)
x_train_unawareness.drop('marital_divorced', inplace=True, axis=1)

# x_train_unawareness.columns.values
# x_train.columns.values






In [36]:
#Fairness through unawareness

x_test_unawareness = x_test.copy()
x_test_unawareness
x_test_married = x_test_unawareness[x_test_unawareness['marital_married'] == 1]
x_test_single = x_test_unawareness[x_test_unawareness['marital_single'] == 1]
x_test_divorced = x_test_unawareness[x_test_unawareness['marital_divorced'] == 1]


x_test_married.drop('marital_married', inplace=True, axis=1)
x_test_married.drop('marital_single', inplace=True, axis=1)
x_test_married.drop('marital_divorced', inplace=True, axis=1)

x_test_single.drop('marital_married', inplace=True, axis=1)
x_test_single.drop('marital_single', inplace=True, axis=1)
x_test_single.drop('marital_divorced', inplace=True, axis=1)

x_test_divorced.drop('marital_married', inplace=True, axis=1)
x_test_divorced.drop('marital_single', inplace=True, axis=1)
x_test_divorced.drop('marital_divorced', inplace=True, axis=1)




# x_test_unawareness.columns.values
# x_test.columns.values
# test_married

/tmp/ipykernel_105020/817814040.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_married.drop('marital_married', inplace=True, axis=1)
/tmp/ipykernel_105020/817814040.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_married.drop('marital_single', inplace=True, axis=1)
/tmp/ipykernel_105020/817814040.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_married.drop('marital_divorced', inplace=True, axis=1)
/

In [37]:
# Test on single set
gnb_pred_single = gnb.fit(x_train_unawareness, y_train).predict(x_test_single)

test_df = x_test_single.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(gnb_pred_single, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_single = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_single



array([[11, 17],
       [ 3,  0]])

In [38]:
accuracy_naive_bayes_single = (confusion_matrix_single[0][0] + confusion_matrix_single[1][1]) / (confusion_matrix_single[0][0] + confusion_matrix_single[0][1] + confusion_matrix_single[1][0] + confusion_matrix_single[1][1])
accuracy_naive_bayes_single 

0.3548387096774194

In [39]:
# Test on married set
gnb_pred_married = gnb.fit(x_train_unawareness, y_train).predict(x_test_married)

test_df = x_test_married.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(gnb_pred_married, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_married = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_married



array([[26, 21],
       [ 5,  4]])

In [40]:
accuracy_naive_bayes_married = (confusion_matrix_married[0][0] + confusion_matrix_married[1][1]) / (confusion_matrix_married[0][0] + confusion_matrix_married[0][1] + confusion_matrix_married[1][0] + confusion_matrix_married[1][1])
accuracy_naive_bayes_married 

0.5357142857142857

In [41]:
# Test on divorced set
gnb_pred_divorced = gnb.fit(x_train_unawareness, y_train).predict(x_test_divorced)

test_df = x_test_divorced.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(gnb_pred_divorced, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_divorced = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_divorced

array([[9, 4],
       [0, 0]])

In [42]:
accuracy_naive_bayes_divorced = (confusion_matrix_divorced[0][0] + confusion_matrix_divorced[1][1]) / (confusion_matrix_divorced[0][0] + confusion_matrix_divorced[0][1] + confusion_matrix_divorced[1][0] + confusion_matrix_divorced[1][1])
accuracy_naive_bayes_divorced # 0.8718606782768102

0.6923076923076923

Random Forest Model

In [43]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=2, random_state=1234, class_weight='balanced')
rf_pred = rf.fit(x_train, y_train).predict(x_test)

In [44]:
test_df = x_test.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(rf_pred, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])
test_df.columns.values

array(['balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'biDefault', 'biHousing', 'biLoan', 'catAge_30-40', 'catAge_40-50',
       'catAge_<=30', 'catAge_>=50', 'job_admin.', 'job_blue-collar',
       'job_entrepreneur', 'job_housemaid', 'job_management',
       'job_retired', 'job_self-employed', 'job_services', 'job_student',
       'job_technician', 'job_unemployed', 'marital_divorced',
       'marital_married', 'marital_single', 'education_primary',
       'education_secondary', 'education_tertiary', 'month_apr',
       'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul',
       'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
       'month_sep', 'Fmonth_apr', 'Fmonth_aug', 'Fmonth_dec',
       'Fmonth_feb', 'Fmonth_jan', 'Fmonth_jul', 'Fmonth_jun',
       'Fmonth_mar', 'Fmonth_may', 'Fmonth_nov', 'Fmonth_oct', 'biY',
       'pred', 'accurate'], dtype=object)

In [45]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix

array([[70, 18],
       [ 9,  3]])

In [46]:
# Accuracy
# "Accuracy: ", test_df.accurate.mean()
accuracy_rf = (confusion_matrix[0][0] + confusion_matrix[1][1]) / (confusion_matrix[0][0] + confusion_matrix[0][1] + confusion_matrix[1][0] + confusion_matrix[1][1])
accuracy_rf


0.73

In [47]:
# Split dataset based on marital status
test_df.columns.values
test_married = x_test[x_test['marital_married'] == 1]
test_single = x_test[x_test['marital_single'] == 1]
test_divorced = x_test[x_test['marital_divorced'] == 1]

In [48]:
# Test on single subset
rf_pred_single = rf.fit(x_train, y_train).predict(test_single)

test_df = test_single.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(rf_pred_single, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_single = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_single

array([[20,  8],
       [ 3,  0]])

In [49]:
accuracy_rf_single = (confusion_matrix_single[0][0] + confusion_matrix_single[1][1]) / (confusion_matrix_single[0][0] + confusion_matrix_single[0][1] + confusion_matrix_single[1][0] + confusion_matrix_single[1][1])
accuracy_rf_single

0.6451612903225806

In [50]:
# Test on married subset
rf_pred_married = rf.fit(x_train, y_train).predict(test_married)

test_df = test_married.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(rf_pred_married, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_married = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_married

array([[38,  9],
       [ 6,  3]])

In [51]:
accuracy_rf_married = (confusion_matrix_married[0][0] + confusion_matrix_married[1][1]) / (confusion_matrix_married[0][0] + confusion_matrix_married[0][1] + confusion_matrix_married[1][0] + confusion_matrix_married[1][1])
accuracy_rf_married 

0.7321428571428571

In [52]:
# Test on divorced subset
rf_pred_divorced = rf.fit(x_train, y_train).predict(test_divorced)
 
test_df = test_divorced.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(rf_pred_divorced, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])
 
from sklearn.metrics import confusion_matrix
 
confusion_matrix_divorced = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_divorced


array([[12,  1],
       [ 0,  0]])

In [53]:
accuracy_naive_bayes_divorced = (confusion_matrix_divorced[0][0] + confusion_matrix_divorced[1][1]) / (confusion_matrix_divorced[0][0] + confusion_matrix_divorced[0][1] + confusion_matrix_divorced[1][0] + confusion_matrix_divorced[1][1])
accuracy_naive_bayes_divorced 

0.9230769230769231

In [54]:
#Demographic Parity
# positive rate = TP + TN
PR_rf_married = confusion_matrix_married[0,0] +confusion_matrix_married[1,1]
PR_rf_married # 4756

41

In [55]:
PR_rf_single = confusion_matrix_single[0,0] +confusion_matrix_single[1,1]
PR_rf_single # 2136

20

In [56]:
PR_rf_divorced = confusion_matrix_divorced[0,0] +confusion_matrix_divorced[1,1]
PR_rf_divorced # 2136

12

In [57]:
# Equalized Opportunity
#TPR = TP/TP+FN

TPR_rf_married = confusion_matrix_married[0,0] / (confusion_matrix_married[0,0] + confusion_matrix_married[1,0])

TPR_rf_married 

0.8636363636363636

In [58]:
TPR_rf_single = confusion_matrix_single[0,0] / (confusion_matrix_single[0,0] + confusion_matrix_single[1,0])

TPR_rf_single 

0.8695652173913043

In [59]:
TPR_rf_divorced = confusion_matrix_divorced[0,0] / (confusion_matrix_divorced[0,0] + confusion_matrix_divorced[1,0])

TPR_rf_divorced 

1.0

In [60]:
# Equalized Odds
#TPR = TP/TP+FN
#FNR = FN/FN+TP


FNR_rf_married = confusion_matrix_married[1,0] / (confusion_matrix_married[1,0] + confusion_matrix_married[0,0])

FNR_rf_married # 0.11652109273369263


0.13636363636363635

In [61]:
FNR_rf_single = confusion_matrix_single[1,0] / (confusion_matrix_single[1,0] + confusion_matrix_single[0,0])

FNR_rf_single 

0.13043478260869565

In [62]:
FNR_rf_divorced = confusion_matrix_divorced[1,0] / (confusion_matrix_divorced[1,0] + confusion_matrix_divorced[0,0])

FNR_rf_divorced 

0.0

In [63]:
# Fairness Through Unwareness
x_train_unawareness = x_train.copy()
x_train_unawareness
x_train_unawareness.drop('marital_married', inplace=True, axis=1)
x_train_unawareness.drop('marital_single', inplace=True, axis=1)
x_train_unawareness.drop('marital_divorced', inplace=True, axis=1)

# x_train_unawareness.columns.values
# x_train.columns.values

In [64]:
#Fairness through unawareness

x_test_unawareness = x_test.copy()
x_test_unawareness
x_test_married = x_test_unawareness[x_test_unawareness['marital_married'] == 1]
x_test_single = x_test_unawareness[x_test_unawareness['marital_single'] == 1]
x_test_divorced = x_test_unawareness[x_test_unawareness['marital_divorced'] == 1]


x_test_married.drop('marital_married', inplace=True, axis=1)
x_test_married.drop('marital_single', inplace=True, axis=1)
x_test_married.drop('marital_divorced', inplace=True, axis=1)

x_test_single.drop('marital_married', inplace=True, axis=1)
x_test_single.drop('marital_single', inplace=True, axis=1)
x_test_single.drop('marital_divorced', inplace=True, axis=1)

x_test_divorced.drop('marital_married', inplace=True, axis=1)
x_test_divorced.drop('marital_single', inplace=True, axis=1)
x_test_divorced.drop('marital_divorced', inplace=True, axis=1)


# x_test_unawareness.columns.values
# x_test.columns.values

/tmp/ipykernel_105020/1113718184.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_married.drop('marital_married', inplace=True, axis=1)
/tmp/ipykernel_105020/1113718184.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_married.drop('marital_single', inplace=True, axis=1)
/tmp/ipykernel_105020/1113718184.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_married.drop('marital_divorced', inplace=True, axis=1

In [65]:
# Test on single set
rf_pred_single = rf.fit(x_train_unawareness, y_train).predict(x_test_single)

test_df = x_test_single.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(rf_pred_single, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_single = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_single


array([[20,  8],
       [ 3,  0]])

In [66]:
accuracy_rf_single = (confusion_matrix_single[0][0] + confusion_matrix_single[1][1]) / (confusion_matrix_single[0][0] + confusion_matrix_single[0][1] + confusion_matrix_single[1][0] + confusion_matrix_single[1][1])
accuracy_rf_single 

0.6451612903225806

In [67]:
# Test on married set
rf_pred_married = rf.fit(x_train_unawareness, y_train).predict(x_test_married)

test_df = x_test_married.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(rf_pred_married, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_married = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_married


array([[36, 11],
       [ 7,  2]])

In [68]:
accuracy_rf_married = (confusion_matrix_married[0][0] + confusion_matrix_married[1][1]) / (confusion_matrix_married[0][0] + confusion_matrix_married[0][1] + confusion_matrix_married[1][0] + confusion_matrix_married[1][1])
accuracy_rf_married 

0.6785714285714286

In [69]:
# Test on divorced set
rf_pred_divorced = rf.fit(x_train_unawareness, y_train).predict(x_test_divorced)

test_df = x_test_divorced.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(rf_pred_divorced, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_divorced = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_divorced

array([[12,  1],
       [ 0,  0]])

In [70]:
accuracy_rf_divorced = (confusion_matrix_divorced[0][0] + confusion_matrix_divorced[1][1]) / (confusion_matrix_divorced[0][0] + confusion_matrix_divorced[0][1] + confusion_matrix_divorced[1][0] + confusion_matrix_divorced[1][1])
accuracy_rf_divorced # 0.8718606782768102

0.9230769230769231

Support Vector Machine

In [71]:
from sklearn import svm as svm_model
svm = svm_model.SVC(class_weight='balanced') # add class_weight='balanced' is to avoid classifying all data as 0
svm_pred = svm.fit(x_train, y_train).predict(x_test)

In [72]:
# from sklearn import svm as svm_model
# svm = svm_model.SVC(class_weight='balanced') # add class_weight='balanced' is to avoid classifying all data as 0
# svm_pred = svm.fit(x_train[:100], y_train[:100]).predict(x_test)

In [73]:
test_df = x_test.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(svm_pred, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])
test_df.columns.values

array(['balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'biDefault', 'biHousing', 'biLoan', 'catAge_30-40', 'catAge_40-50',
       'catAge_<=30', 'catAge_>=50', 'job_admin.', 'job_blue-collar',
       'job_entrepreneur', 'job_housemaid', 'job_management',
       'job_retired', 'job_self-employed', 'job_services', 'job_student',
       'job_technician', 'job_unemployed', 'marital_divorced',
       'marital_married', 'marital_single', 'education_primary',
       'education_secondary', 'education_tertiary', 'month_apr',
       'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul',
       'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
       'month_sep', 'Fmonth_apr', 'Fmonth_aug', 'Fmonth_dec',
       'Fmonth_feb', 'Fmonth_jan', 'Fmonth_jul', 'Fmonth_jun',
       'Fmonth_mar', 'Fmonth_may', 'Fmonth_nov', 'Fmonth_oct', 'biY',
       'pred', 'accurate'], dtype=object)

In [74]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix

array([[75, 13],
       [ 7,  5]])

In [75]:
# Accuracy
# "Accuracy: ", test_df.accurate.mean()
accuracy_svm = (confusion_matrix[0][0] + confusion_matrix[1][1]) / (confusion_matrix[0][0] + confusion_matrix[0][1] + confusion_matrix[1][0] + confusion_matrix[1][1])
accuracy_svm


0.8

In [76]:
# Split dataset based on marital status
test_df.columns.values
test_married = x_test[x_test['marital_married'] == 1]
test_single = x_test[x_test['marital_single'] == 1]
test_divorced = x_test[x_test['marital_divorced'] == 1]

In [77]:
# Test on single subset
svm_pred_single = svm.fit(x_train, y_train).predict(test_single)

test_df = test_single.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(svm_pred_single, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_single = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_single

array([[24,  4],
       [ 1,  2]])

In [78]:
accuracy_svm_single = (confusion_matrix_single[0][0] + confusion_matrix_single[1][1]) / (confusion_matrix_single[0][0] + confusion_matrix_single[0][1] + confusion_matrix_single[1][0] + confusion_matrix_single[1][1])
accuracy_svm_single

0.8387096774193549

In [79]:
# Test on married subset
svm_pred_married = svm.fit(x_train, y_train).predict(test_married)

test_df = test_married.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(svm_pred_married, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_married = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_married

array([[41,  6],
       [ 6,  3]])

In [80]:
accuracy_svm_married = (confusion_matrix_married[0][0] + confusion_matrix_married[1][1]) / (confusion_matrix_married[0][0] + confusion_matrix_married[0][1] + confusion_matrix_married[1][0] + confusion_matrix_married[1][1])
accuracy_svm_married # 0.8718606782768102

0.7857142857142857

In [81]:
# Test on divorced subset
svm_pred_divorced = svm.fit(x_train, y_train).predict(test_divorced)

test_df = test_divorced.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(svm_pred_divorced, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_divorced = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_divorced

array([[10,  3],
       [ 0,  0]])

In [82]:
accuracy_svm_divorced = (confusion_matrix_divorced[0][0] + confusion_matrix_divorced[1][1]) / (confusion_matrix_divorced[0][0] + confusion_matrix_divorced[0][1] + confusion_matrix_divorced[1][0] + confusion_matrix_divorced[1][1])
accuracy_svm_divorced # 0.8718606782768102

0.7692307692307693

In [83]:
#Demographic Parity
# positive rate = TP + TN
PR_svm_married = confusion_matrix_married[0,0] +confusion_matrix_married[1,1]

PR_svm_married

44

In [84]:
PR_svm_single = confusion_matrix_single[0,0] +confusion_matrix_single[1,1]
PR_svm_single

26

In [85]:
PR_svm_divorced = confusion_matrix_divorced[0,0] +confusion_matrix_divorced[1,1]
PR_svm_divorced 

10

In [86]:
# Equalized Opportunity
#TPR = TP/TP+FN

TPR_svm_married = confusion_matrix_married[0,0] / (confusion_matrix_married[0,0] + confusion_matrix_married[1,0])

TPR_svm_married 

0.8723404255319149

In [87]:
TPR_svm_single = confusion_matrix_single[0,0] / (confusion_matrix_single[0,0] + confusion_matrix_single[1,0])

TPR_svm_single 

0.96

In [88]:
TPR_svm_divorced = confusion_matrix_divorced[0,0] / (confusion_matrix_divorced[0,0] + confusion_matrix_divorced[1,0])

TPR_svm_divorced 

1.0

In [89]:
# Equalized Odds
#TPR = TP/TP+FN
#FNR = FN/FN+TP


FNR_svm_married = confusion_matrix_married[1,0] / (confusion_matrix_married[1,0] + confusion_matrix_married[0,0])

FNR_svm_married

0.1276595744680851

In [90]:
FNR_svm_single = confusion_matrix_single[1,0] / (confusion_matrix_single[1,0] + confusion_matrix_single[0,0])

FNR_svm_single 

0.04

In [91]:
FNR_svm_divorced = confusion_matrix_divorced[1,0] / (confusion_matrix_divorced[1,0] + confusion_matrix_divorced[0,0])

FNR_svm_divorced 

0.0

In [92]:
# Fairness Through Unwareness
x_train_unawareness = x_train.copy()
x_train_unawareness
x_train_unawareness.drop('marital_married', inplace=True, axis=1)
x_train_unawareness.drop('marital_single', inplace=True, axis=1)
x_train_unawareness.drop('marital_divorced', inplace=True, axis=1)

# x_train_unawareness.columns.values
# x_train.columns.values

In [93]:
#Fairness through unawareness

x_test_unawareness = x_test.copy()
x_test_unawareness
x_test_married = x_test_unawareness[x_test_unawareness['marital_married'] == 1]
x_test_single = x_test_unawareness[x_test_unawareness['marital_single'] == 1]
x_test_divorced = x_test_unawareness[x_test_unawareness['marital_divorced'] == 1]


x_test_married.drop('marital_married', inplace=True, axis=1)
x_test_married.drop('marital_single', inplace=True, axis=1)
x_test_married.drop('marital_divorced', inplace=True, axis=1)

x_test_single.drop('marital_married', inplace=True, axis=1)
x_test_single.drop('marital_single', inplace=True, axis=1)
x_test_single.drop('marital_divorced', inplace=True, axis=1)

x_test_divorced.drop('marital_married', inplace=True, axis=1)
x_test_divorced.drop('marital_single', inplace=True, axis=1)
x_test_divorced.drop('marital_divorced', inplace=True, axis=1)



/tmp/ipykernel_105020/2448254121.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_married.drop('marital_married', inplace=True, axis=1)
/tmp/ipykernel_105020/2448254121.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_married.drop('marital_single', inplace=True, axis=1)
/tmp/ipykernel_105020/2448254121.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_married.drop('marital_divorced', inplace=True, axis=1

In [94]:
# Test on single set
svm_pred_single = svm.fit(x_train_unawareness, y_train).predict(x_test_single)

test_df = x_test_single.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(svm_pred_single, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_single = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_single


array([[24,  4],
       [ 1,  2]])

In [95]:
accuracy_svm_single = (confusion_matrix_single[0][0] + confusion_matrix_single[1][1]) / (confusion_matrix_single[0][0] + confusion_matrix_single[0][1] + confusion_matrix_single[1][0] + confusion_matrix_single[1][1])
accuracy_svm_single

0.8387096774193549

In [96]:
# Test on married set
svm_pred_married = svm.fit(x_train_unawareness, y_train).predict(x_test_married)

test_df = x_test_married.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(svm_pred_married, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_married = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_married

array([[41,  6],
       [ 6,  3]])

In [97]:
accuracy_svm_married = (confusion_matrix_married[0][0] + confusion_matrix_married[1][1]) / (confusion_matrix_married[0][0] + confusion_matrix_married[0][1] + confusion_matrix_married[1][0] + confusion_matrix_married[1][1])
accuracy_svm_married # 0.8718606782768102

0.7857142857142857

In [98]:
# Test on divorced subset
svm_pred_divorced = svm.fit(x_train_unawareness, y_train).predict(x_test_divorced)

test_df = x_test_divorced.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(svm_pred_divorced, index=test_df.index)
test_df['accurate'] = (test_df['pred'] == test_df['biY'])

from sklearn.metrics import confusion_matrix

confusion_matrix_divorced = confusion_matrix(test_df['biY'], test_df['pred'])
confusion_matrix_divorced

array([[10,  3],
       [ 0,  0]])

In [99]:
accuracy_svm_divorced = (confusion_matrix_divorced[0][0] + confusion_matrix_divorced[1][1]) / (confusion_matrix_divorced[0][0] + confusion_matrix_divorced[0][1] + confusion_matrix_divorced[1][0] + confusion_matrix_divorced[1][1])
accuracy_svm_divorced 

0.7692307692307693

Linear Regression model

In [100]:
from sklearn import linear_model
linear_regression_model = linear_model.LinearRegression()
linear_regression_model.fit(x_train, y_train)
prediction = linear_regression_model.predict(x_test)

Result

In [101]:
test_df = x_test.copy()
test_df['biY'] = y_test
test_df['pred'] = pd.Series(prediction, index=test_df.index)

test_df['accurate'] = (test_df['pred'] == test_df['biY'])
test_df['pred']

67     0.107286
416    0.093484
350    0.172181
358    0.036950
112    0.037081
         ...   
214    0.073261
181    0.177972
386    0.163523
407    0.021476
435    0.097750
Name: pred, Length: 100, dtype: float64